# LSMS Linear Regression 
Predict farm-level yields using vegetation index measures.

In [1]:
import pandas as pd
import statsmodels.api as sm

In [2]:
df = pd.read_csv('Apr_2015_LSMS_VI_EE_Export - wm_export_test.csv')

In [3]:
print(f"There are {df.shape[0]} observations prior to removing NAs")

There are 25496 observations prior to removing NAs


In [4]:
reg_vars = ['Quantity (kg)','NDVI_mean','NDVI_variance','TVI_mean','TVI_variance','mean_EVI']
df = df[reg_vars]
df = df.dropna()

In [5]:
y = df[['Quantity (kg)']]
x = df[['NDVI_mean','NDVI_variance','TVI_mean','TVI_variance','mean_EVI']]

In [6]:
print(f"There are {df.shape[0]} observations after removing NAs")

There are 23683 observations after removing NAs


In [7]:
mod = sm.OLS(y,x)

res = mod.fit()

print(res.summary())

                                 OLS Regression Results                                
Dep. Variable:          Quantity (kg)   R-squared (uncentered):                   0.241
Model:                            OLS   Adj. R-squared (uncentered):              0.241
Method:                 Least Squares   F-statistic:                              1505.
Date:                Tue, 01 Oct 2019   Prob (F-statistic):                        0.00
Time:                        14:17:08   Log-Likelihood:                     -1.5366e+05
No. Observations:               23683   AIC:                                  3.073e+05
Df Residuals:                   23678   BIC:                                  3.074e+05
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
                    coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------